# Introdução ao ETL em Python

In [16]:
# Import the required library
import requests

path = 'https://github.com/viniciusperdigao/Engenharia-de-Dados/raw/main/ETL%20em%20Python/base/PPR-2021.zip'
# Get the zip file
response = requests.get(path)

# Print the status code
print(response.status_code)

#Save the file locally 
local_path = "./PPR-ALL.zip"
with open(local_path, "wb") as f:
    f.write(response.content)

200


Acabamos de baixar a fonte dos dados e armazenamos em um diretório local. Mas o que realmente precisamos é do arquivo que ele contém: um CSV contendo dados sobre as transações imobiliárias e suas características.

In [21]:
# Import the required method
from zipfile import ZipFile

path = './PPR-ALL.zip'

with ZipFile(path, mode="r") as f:
  	# Get the list of files and print it
    file_names = f.namelist()
    print(file_names)
    # Extract the CSV file
    csv_file_path = f.extract(file_names[0])
    print(csv_file_path)

['PPR-2021.csv']
